### Look at units most highly correlated with feat entropy

In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.behavioral_utils as behavioral_utils
import utils.information_utils as information_utils
import utils.visualization_utils as visualization_utils
import utils.glm_utils as glm_utils
from matplotlib import pyplot as plt
import utils.spike_utils as spike_utils
from constants.glm_constants import *
from constants.behavioral_constants import *
from spike_tools import (
    general as spike_general,
    analysis as spike_analysis,
)
from scipy import stats
import warnings
from scipy.ndimage import gaussian_filter1d
import seaborn as sns

warnings.filterwarnings('ignore')

In [4]:
correlations = pd.read_pickle("/data/patrick_res/glm_2/feat_entropy_correlations.pickle")
SESSIONS_PATH = "/data/patrick_res/sessions/valid_sessions_rpe.pickle"
sessions = pd.read_pickle(SESSIONS_PATH)
positions = spike_utils.get_unit_positions(sessions)
correlations = pd.merge(correlations, positions, on="PseudoUnitID")

In [6]:
correlations.sort_values(by="r_value")[:10][["PseudoUnitID", "r_value", "structure_level2"]]

,PseudoUnitID,r_value,structure_level2
656,2018071202,-0.572822,basal_ganglia (BG)
537,2018101008,-0.504193,basal_ganglia (BG)
753,2018091027,-0.386860,basal_ganglia (BG)
325,2018080200,-0.369936,anterior_cingulate_gyrus (ACgG)
522,2018100909,-0.367191,basal_ganglia (BG)
701,2018071247,-0.364948,amygdala (Amy)
588,2018092505,-0.350980,inferior_parietal_lobule (IPL)
733,2018091007,-0.331792,medial_pallium (MPal)
349,2018080224,-0.322300,lateral_prefrontal_cortex (lat_PFC)
509,2018100315,-0.315683,inferior_parietal_lobule (IPL)


In [10]:
correlations[correlations.p_value < 0.01]

,UnitID_x,session_x,PseudoUnitID,slope,intercept,r_value,p_value,std_err,Channel,Unit,...,tissue,structure_level1,structure_level2,structure_level3,structure_level4,structure_level5,structure_level6,structure_potential,session_y,manual_structure
1,1,20180709,2018070901,-11.117370,27.645319,-0.121419,2.872667e-04,3.053339,108,1,...,Subcortical_gray_matter,telencephalon (tel),lateral_and_ventral_pallium (LVPal),lateral_pallium (LPal),claustrum (Cl),claustrum (Cl),claustrum (Cl),[],20180709,Claustrum
6,6,20180709,2018070906,35.868535,-85.242775,0.100815,2.632772e-03,11.891975,12,1,...,Cortical_gray_matter,Occipital_Lobe (Occipital),primary_visual_cortex (V1),primary_visual_cortex (V1),primary_visual_cortex (V1),primary_visual_cortex (V1),primary_visual_cortex (V1),[],20180709,Visual Cortex
7,7,20180709,2018070907,23.993359,-59.015298,0.134095,6.112376e-05,5.956914,122,1,...,Subcortical_gray_matter,telencephalon (tel),lateral_and_ventral_pallium (LVPal),lateral_pallium (LPal),claustrum (Cl),claustrum (Cl),claustrum (Cl),[],20180709,Claustrum
8,8,20180709,2018070908,120.489102,-294.905155,0.302491,3.023136e-20,12.755016,15a,1,...,Cortical_gray_matter,Frontal_Lobe (Frontal),lateral_prefrontal_cortex (lat_PFC),dorsolateral_prefrontal_cortex (dlPFC),rostral_dorsolateral_prefrontal_cortex (rostra...,area_9 (area_9),medial_area_9 (area_9m),dlPFC,20180709,Prefrontal Cortex
10,10,20180709,2018070910,154.561612,-368.429874,0.129654,1.068383e-04,39.711744,25a,1,...,Cortical_gray_matter,Frontal_Lobe (Frontal),lateral_prefrontal_cortex (lat_PFC),dorsolateral_prefrontal_cortex (dlPFC),rostral_dorsolateral_prefrontal_cortex (rostra...,dorsal_area_46 (area_46d),dorsal_area_46 (area_46d),dlPFC,20180709,Prefrontal Cortex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,10,20180910,2018091010,84.560260,-207.938750,0.196575,6.384314e-09,14.415931,28a,2,...,Cortical_gray_matter,Frontal_Lobe (Frontal),lateral_prefrontal_cortex (lat_PFC),dorsolateral_prefrontal_cortex (dlPFC),rostral_dorsolateral_prefrontal_cortex (rostra...,dorsal_area_46 (area_46d),dorsal_area_46 (area_46d),dlPFC,20180910,Prefrontal Cortex
739,13,20180910,2018091013,386.172930,-951.983795,0.433661,1.180360e-40,27.425566,38,1,...,Cortical_gray_matter,Temporal_Lobe (Temporal),inferior_temporal_cortex (ITC),area_TE (TE),gyral_aspect_of_area_TE (gyral_TE),posterior_area_TE (post_TE),posterior_ventral_area_TE (TEpv),Hippocampal,20180910,Hippocampus/MTL
743,17,20180910,2018091017,74.286417,-181.298480,0.176033,2.110119e-07,14.198485,54a,1,...,Cortical_gray_matter,Frontal_Lobe (Frontal),lateral_prefrontal_cortex (lat_PFC),ventrolateral_prefrontal_cortex (vlPFC),rostral_ventrolateral_prefrontal_cortex (rostr...,ventral_and_fundus_portion_of_area_46 (area_46...,ventral_area_46 (area_46v),[],20180910,Prefrontal Cortex
748,22,20180910,2018091022,200.077943,-484.740901,0.233974,3.919894e-12,28.416443,77,1,...,Cortical_gray_matter,Temporal_Lobe (Temporal),floor_of_the_lateral_sulcus (floor_of_ls),floor_of_the_lateral_sulcus (floor_of_ls),insula_and_parainsula (Ins/Pi),insula (Ins),granular_insula (Ig),[],20180910,unknown
